In [1]:
# !pip install transformers[sentencepiece]
# !python -m spacy download en_core_web_lg
# !pip install textpipeliner

In [2]:
sent_list = ["The three different ways to become a brain dead zombie are, a virus, radiation and Chrisitanity.", 
             "A gay killing another gay nice.", 
             "...I'm that fast. Hitting someone going full speed. Over 20 mph 190 lbs. I'll kill them.",
             "I'm gay but I've got the biggest fucking crush on @katyperry I have for years.", 
             "Donald Trump is the worst president of USA, but Hillary is better than him.", 
             "Was on an absolute heater, then I lose 500 dollars on Korean baseball and Costa Riccan soccer. I hate my life.",
             "Once at work, I accidentally checked out some dragons fucking cars. Did not know that would be an issue.",
             "The girl on the left looks like a Jewish pizza.",
             "The Blacks and Browns are destroying Sweden and nobody does anything about it!",
             "I’ve been out for years and it hasn’t been easy, but I also love being a fag.",
             "Why do you not think so? Feck off. Please go on. You annoy me.",
             "All we have is Blacks and a few Mestizos and all are pretty well behaved, we keep them that way around here.",
             "I truly am a badass motherfucker.",
             "Oh yeah, she is so annoying I had mentally blocked out her existence.",
             "As opposed to Reddit, which is composed mainly of Communists jacking each other off about child drag queens?",
             "KPOP is so gay, but the day is so beautiful today.",
             "Asians are the worst race, but Americans are soooo good.",
             "Asians are the best race, but Americans are soooo gross.",
             "I'd kill all the Blacks, but my mom raised me up to be such a nice human."]

In [3]:
import nltk 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def extract_NN(sent):
    grammar = r"""
    NBAR:
        # Nouns and Adjectives, terminated with Nouns
        {<NN.*>*<NN.*>}

    NP:
        {<NBAR>}
        # Above, connected with in/of/etc...
        {<NBAR><IN><NBAR>}
    """
    chunker = nltk.RegexpParser(grammar)
    ne = set()
    chunk = chunker.parse(nltk.pos_tag(nltk.word_tokenize(sent)))
    for tree in chunk.subtrees(filter=lambda t: t.label() == 'NP'):
        ne.add(' '.join([child[0] for child in tree.leaves()]).lower())
    return ne

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\psheth5\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\psheth5\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
for sent in sent_list:
  print(sent, " --> ",extract_NN(sent))

The three different ways to become a brain dead zombie are, a virus, radiation and Chrisitanity.  -->  {'ways', 'chrisitanity', 'brain', 'virus', 'zombie', 'radiation'}
A gay killing another gay nice.  -->  {'gay', 'gay nice'}
...I'm that fast. Hitting someone going full speed. Over 20 mph 190 lbs. I'll kill them.  -->  {'fast', 'someone', 'lbs', 'mph', 'speed'}
I'm gay but I've got the biggest fucking crush on @katyperry I have for years.  -->  {'@ katyperry', 'years', 'fucking crush'}
Donald Trump is the worst president of USA, but Hillary is better than him.  -->  {'donald trump', 'president', 'usa', 'hillary'}
Was on an absolute heater, then I lose 500 dollars on Korean baseball and Costa Riccan soccer. I hate my life.  -->  {'life', 'was', 'dollars', 'korean baseball', 'costa riccan soccer', 'heater'}
Once at work, I accidentally checked out some dragons fucking cars. Did not know that would be an issue.  -->  {'issue', 'work', 'dragons', 'cars', 'did'}
The girl on the left looks 

In [5]:
import spacy
nlp = spacy.load('en_core_web_lg')

C:\Users\psheth5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import torch.nn.functional as F
from transformers import pipeline
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-large-absa-v1.1")
absa_model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-large-absa-v1.1")

absa_model.to(device=device)

# # Load a traditional Sentiment Analysis model
# sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
# sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,tokenizer=sentiment_model_path)

task='sentiment'
latest_task='sentiment-latest'
sentiment_MODEL = f"cardiffnlp/twitter-roberta-base-{task}"#This is a roBERTa-base model trained on ~58M
sentiment_MODEL_new_SA = f"cardiffnlp/twitter-roberta-base-{latest_task}"#This is a roBERTa-base model trained on ~124M
# sentiment_MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
# sentiment_MODEL_new_SA = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
#we will pick what we need and what helps us achive better results
tokenizer = AutoTokenizer.from_pretrained(sentiment_MODEL)
tokenizer_new_SA = AutoTokenizer.from_pretrained(sentiment_MODEL_new_SA)

#print("###########tokenizer###########")
#tokenizer
#print("############config##########")
# waht is this config = AutoConfig.from_pretrained(sentiment_MODEL) learn more about it 
config = AutoConfig.from_pretrained(sentiment_MODEL)
config_new_SA = AutoConfig.from_pretrained(sentiment_MODEL)

sentiment_model_new_SA = AutoModelForSequenceClassification.from_pretrained(sentiment_MODEL)
sentiment_model_new_SA.save_pretrained(sentiment_MODEL)
sentiment_model_new_SA.to(device=device)

C:\Users\psheth5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [7]:
def get_aspect_sentiment(aspect,sentence):
  inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", add_special_tokens=True,
    max_length=512,padding="max_length" , truncation=True, return_tensors="pt")
  inputs.to(device)
  outputs = absa_model(**inputs)
  probs = F.softmax(outputs.logits, dim=1)
  probs = probs.detach().cpu().numpy()[0]
  out = np.zeros_like(probs)
  idx  = probs.argmax()
  out[idx] = 1
  return out


  return probs

In [8]:
def get_sentiment(sentence):

  encoded_input = tokenizer_new_SA(sentence,add_special_tokens=True,
    max_length=512,padding="max_length" , truncation=True, return_tensors='pt') #i commented and added ,padding=True
  encoded_input.to(device)#mostly stuck here the arrow
  output = sentiment_model_new_SA(**encoded_input)

  probs = F.softmax(output.logits, dim=1)
  
  probs = probs.detach().cpu().numpy()[0]
  
  out = np.zeros_like(probs)
  idx  = probs.argmax()
  out[idx] = 1
  return out

In [9]:
def get_subject_phrase(doc):
    res = []
    for token in doc:
        if (token.dep_ in ["agent", "expl","dobj", "dative", "oprd"] and token.text not in res and token.text.strip() not in ["I","me","my","myself"]):
            res.append(token.text.strip())
    return res

In [10]:
def get_target(doc):
  temp=[]
  for tok in doc:
    if(tok.tag_ in ["PRP","NNP","NNPS"] and tok.text not in temp and tok.text.strip() not in ["i","ME","MY","MYSELF","Me","My","Myself","I","me","my","myself"]):
      temp.append(tok.text.strip().lower())
  return temp

In [11]:
def extract_overall_sentiment_features(data_frame):
    feature_list = []
    for row in tqdm(data_frame.itertuples()):

        feature_row = []

        sentence = row.text
        doc=nlp(sentence)
        avg_sentiment = np.zeros(3)
        sent = get_sentiment(sentence)
        avg_sentiment = sent
        feature_row.append(sentence)
        feature_row.append(row.label)
        feature_row.extend(avg_sentiment)

        feature_list.append(feature_row)
    
    feature_frame = pd.DataFrame(feature_list, columns = ["text", "label", "sent_neg", "sent_neut", "sent_pos"])
    return feature_frame

In [12]:
from tqdm import tqdm 
import numpy as np

def extract_features(data_frame):
  feature_list = []

  for row in tqdm(data_frame.itertuples()):

    feature_row = []

    sentence = row.text
    doc=nlp(sentence)
    phrases1 = get_target(doc)
    phrases2 = get_subject_phrase(doc)
    phrases1.extend(phrases2)
    phrases = list(set(phrases1))
    # print(phrases)

    # phrases = []
    avg_sentiment = np.zeros(3)
    if(len(phrases)==0):
      sent = get_sentiment(sentence)
      avg_sentiment = sent
    else:
      for phrase in phrases:
        avg_sentiment+=np.array(get_aspect_sentiment(phrase,sentence)/len(phrases))
    
    feature_row.append(sentence)
    feature_row.append(row.label)
    feature_row.extend(avg_sentiment)

    feature_list.append(feature_row)
    # print(feature_row)
  
  feature_frame = pd.DataFrame(feature_list, columns = ["text", "label", "sent_neg", "sent_neut", "sent_pos"])
  
  return feature_frame

In [13]:
import pandas as pd

lgbt_train_data = pd.read_csv("C:\\Users\\psheth5\\Downloads\\Stylometric-emotion-approach (1)\\Stylometric-emotion-approach\\lgbt-train.csv")
lgbt_test_data = pd.read_csv("C:\\Users\\psheth5\\Downloads\\Stylometric-emotion-approach (1)\\Stylometric-emotion-approach\\lgbt-test.csv")

migrants_train_data = pd.read_csv("C:\\Users\\psheth5\\Downloads\\Stylometric-emotion-approach (1)\\Stylometric-emotion-approach\\migrants-train.csv")
migrants_test_data = pd.read_csv("C:\\Users\\psheth5\\Downloads\\Stylometric-emotion-approach (1)\\Stylometric-emotion-approach\\migrants-test.csv")

In [14]:
#  ASPECT BASED EXTRACTION
# lgbt_train_features = extract_features(lgbt_train_data)
# lgbt_test_features = extract_features(lgbt_test_data)

# migrants_train_features = extract_features(migrants_train_data)
# migrants_test_features = extract_features(migrants_test_data)

# #  OVERALL EXTRACTION
lgbt_train_features = extract_overall_sentiment_features(lgbt_train_data)
lgbt_test_features = extract_overall_sentiment_features(lgbt_test_data)

migrants_train_features = extract_overall_sentiment_features(migrants_train_data)
migrants_test_features = extract_overall_sentiment_features(migrants_test_data)

4073it [01:09, 58.46it/s]
1017it [00:16, 60.56it/s]
3866it [01:04, 59.90it/s]
1078it [00:19, 55.79it/s]


In [15]:
lgbt_train_features

,text,label,sent_neg,sent_neut,sent_pos
0,yes.,0,0.0,0.0,1.0
1,Surely a marriage is a ceremony involving two ...,0,0.0,0.0,1.0
2,"We can't help who we fall inlove with, let the...",0,1.0,0.0,0.0
3,Yes...when they will be able to give birth.,1,0.0,1.0,0.0
4,"Society began with Adam and Eve. Nothing more,...",1,0.0,1.0,0.0
...,...,...,...,...,...
4068,"You been punked by the Trump, it was cool unti...",1,1.0,0.0,0.0
4069,When transgenders want to have special treatme...,1,1.0,0.0,0.0
4070,...Ignorant Bigot.....,1,1.0,0.0,0.0
4071,"Devon when you look up the word bigot, its an ...",1,0.0,1.0,0.0


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
# from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

# from sklearn.metrics import precision_score, \
#     recall_score, confusion_matrix, classification_report, \
#     accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support,accuracy_score

def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description):
    
    model = LogisticRegression().fit(X_tr, y_tr)
    
    prediction = model.predict(X_test)
    f1_score, precision, recall, _ = precision_recall_fscore_support(y_test, prediction, average='macro')
    print('Accuracy:', accuracy_score(y_test, prediction))
    print('F1 score:', f1_score)
    print('Recall:', recall)
    print('Precision:', precision)

    return model

# def simple_xgboost_classify(X_tr, y_tr, X_test, y_test, description):
    
#     model = XGBClassifier().fit(X_tr, y_tr)
    
#     prediction = model.predict(X_test)

#     print('Accuracy:', accuracy_score(y_test, prediction))
#     print('F1 score:', f1_score(y_test, prediction))
#     print('Recall:', recall_score(y_test, prediction))
#     print('Precision:', precision_score(y_test, prediction))

#     return model

def simple_randomforest_classify(X_tr, y_tr, X_test, y_test, description):
    
    model = RandomForestClassifier().fit(X_tr, y_tr)
    
    prediction = model.predict(X_test)

    prediction = model.predict(X_test)
    f1_score, precision, recall, _ = precision_recall_fscore_support(y_test, prediction, average='macro')
    print('Accuracy:', accuracy_score(y_test, prediction))
    print('F1 score:', f1_score)
    print('Recall:', recall)
    print('Precision:', precision)

    return model

In [21]:
X_tr = lgbt_train_features[[ "sent_neg", "sent_neut", "sent_pos"]] 
y_tr = lgbt_train_features["label"]

X_test = migrants_test_features[[ "sent_neg", "sent_neut", "sent_pos"]] 
y_test = migrants_test_features["label"]

simple_logistic_classify(X_tr, y_tr, X_test, y_test, description="lr_in_domian")
print()

# simple_xgboost_classify(X_tr, y_tr, X_test, y_test, description="lr_in_domian")
# print()

simple_randomforest_classify(X_tr, y_tr, X_test, y_test, description="lr_in_domian")
print()

Accuracy: 0.525974025974026
F1 score: 0.262987012987013
Recall: 0.3446808510638298
Precision: 0.5

Accuracy: 0.525974025974026
F1 score: 0.262987012987013
Recall: 0.3446808510638298
Precision: 0.5



C:\Users\psheth5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\psheth5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
X_tr = migrants_train_features[[ "sent_neg", "sent_neut", "sent_pos"]] 
y_tr = migrants_train_features["label"]

X_test = lgbt_test_features[[ "sent_neg", "sent_neut", "sent_pos"]] 
y_test = lgbt_test_features["label"]

simple_logistic_classify(X_tr, y_tr, X_test, y_test, description="lr_in_domian")
print()

# simple_xgboost_classify(X_tr, y_tr, X_test, y_test, description="lr_in_domian")
# print()

simple_randomforest_classify(X_tr, y_tr, X_test, y_test, description="lr_in_domian")
print()

Accuracy: 0.6509341199606686
F1 score: 0.6517345399698341
Recall: 0.6236133163332858
Precision: 0.7147784006306386

Accuracy: 0.6509341199606686
F1 score: 0.6517345399698341
Recall: 0.6236133163332858
Precision: 0.7147784006306386



In [22]:
from sklearn.svm import LinearSVC

clf_svc = LinearSVC(random_state=0, C=0.1) # parameter C was selected based on grid search
clf_svc.fit(X_tr, y_tr)
Y_pred = clf_svc.predict(X_test)

results = pd.DataFrame(
    [list(precision_recall_fscore_support(y_test, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results['Accuracy'] = accuracy_score(y_test, Y_pred)
results

C:\Users\psheth5\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,F1,Accuracy
0,0.262987,0.5,0.344681,0.525974


In [20]:
#ORIGINAL SENTIMENT
#           L->M    |   M->L
# Precision  0.263  |   0.651
# Recall     0.500  |   0.623
# F1-Score   0.344  |   0.623
# Accuracy   0.525  |   0.650


#ASPECT BASED SENTIMENT
#           L->M    |   M->L
# Precision  0.596  |   0.663
# Recall     0.554  |   0.686
# F1-Score   0.508  |   0.634
# Accuracy   0.571  |   0.683

#ILIA MARKOV (Full Model)
#           L->M    |   M->L
# Precision  0.669  |   0.582
# Recall     0.620  |   0.588
# F1-Score   0.600  |   0.584
# Accuracy   0.637  |   0.695

#ILIA MARKOV (Sentiment Only)
#           L->M    |   M->L
# Precision  0.488  |   0.509
# Recall     0.499  |   0.506
# F1-Score   0.349  |   0.500
# Accuracy   0.532  |   0.699